Author: Hannah E. Schmidt

In [ ]:
#import packages
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [ ]:
# files that get created in this script
df_for_biclustering_path = "<path to file>" #dataframe for biclustering
for_pheno_enrich_path = "<path to file>" #og: for_loci_enrichment
for_loci_enrich_path = "<path to file>"
overview_CoCluster = "<path to file>" 
final_pheno_loci_df_path = "<path to file>" 

# BiLouvain 
bicluster_path = "<path to directory>" #path where output files from biclustering can be found

# files are output from gene enrichment analysis
enrichment_results_loci = "<path to file>"  #output from gene_enrichment.R
enrichment_results_pheno = "<path to file>" #outpuf from gene_enrichment.R


# 01 Create file for BiLouvain-Clustering

In [ ]:
# import files 

#file containing combined GWAS results, detailed association mapping
loci_file = "<path to file>" 

#file containing LDSC results, containing genetic correlation and heritability analysis
phenotypes_file = "<path to file>" 

# file in "matrix format", contains all phenotypes and all loci, if association in GWAS was found 1 otherwise 0
matrix_file = "<path to file>" 

In [ ]:
# loading data from loci and phenotype file
df_pheno = pd.read_csv(phenotypes_file, sep="\t")
df_loci = pd.read_csv(loci_file, sep="\t")

## derive from matrix file the phenotype names and locis 
def create_array_from_line(line):
    arrays = []
    for entry in line.split(" "):
        if "L" in entry:
            #print("L", entry)
            loci = entry.replace('"', '')
        elif "\n" in entry:
            arrays.append(int(entry.split("\n")[0]))
    
        else:
            arrays.append(int(entry))
    return arrays, loci

f = open(matrix_file)
loci = []
for count, line in enumerate(f.readlines()):
    if count == 0:
        #get names of phenotypes
        phenotypes = line
    else:
        new_array, locus_name = create_array_from_line(line)
        if count == 1:
            matrix = np.array([new_array])
        else:
            matrix = np.append(matrix, [new_array], axis=0)
        loci.append(locus_name)

f.close()
phenotypes = [pheno.replace('"', '') for pheno in phenotypes.split(" ")]
phenotypes = [pheno.replace('\n', '') for pheno in phenotypes]

print("Number of phenotypes:", np.unique(len(phenotypes)))
print("Number of Loci:", np.unique(len(loci)))

In [ ]:
## modify dataframe, only keep columns of interest

locus_df_cols = ["locus_id.r2", "id","ID","category","phecode", "phenotype", "locus_id", "alleleA", "alleleB", "v2g.top","study_id.ot", "trait_reported.gwas", "closest.gene.body", "closest.gene.tss", "closest.genes", "candidate.gene", "candidate.gene.locus.r2"]

df_loci_info = pd.DataFrame()
for locus_id in loci:
    res = df_loci[df_loci["locus_id.r2"]== locus_id[2:]][locus_df_cols]
    df_loci_info = pd.concat([df_loci_info, res])

df_loci_info.reset_index(drop=True, inplace=True)
print("created df_loci_info dataframe")

In [ ]:
df_biclustering = df_loci_info[["locus_id.r2", "id"]].reset_index(drop=True)

#save dataframe for biclustering tool
df_biclustering.to_csv(df_for_biclustering_path, index=False, header=False)

# 02 run BiLouvain Clustering

# 03 create dataframe from BiLouvain Clustering Results

In [ ]:
### import biLouvain files


#bicluster_path: path where output files from biclustering can be found
cluster_dict = bicluster_path + "reformated_links_for_biclustering_bipartite_Dictionary.txt"

cocluster_res = bicluster_path + "reformated_links_for_biclustering_bipartite_ResultsCoClusterCommunities.txt"

communities_res = bicluster_path + "reformated_links_for_biclustering_bipartite_ResultsCommunities.txt"

#init_communities: all loci connected to one phecode build one community, or one loci connected to different phenocodes
init_communities = bicluster_path + "reformated_links_for_biclustering_bipartite_InitialCommunities.txt"


In [ ]:
#read in file
bicluster_dict = pd.read_csv(cluster_dict, sep="\t", header=None)
bicluster_dict.rename(columns={0: "id", 1: "vertices"}, inplace=True)

In [ ]:
#create df of bilouvain community result
f = open(communities_res)
entries = []
cluster_ids = []
for count, line in enumerate(f.readlines()):
    if line == "\n":
        break
    for entry in line.split(":")[1].split(","):
        if entry[0] == " ":
            entry = entry[1:]
        
        if "\n" in entry:
            entries.append(entry.split("\n")[0])
        else:
            entries.append(entry)
        cluster_ids.append(count)   

f.close()

# each locus and phenotype is assigned one cluster/community id
df_communities = pd.DataFrame({"entry": entries, "Community": cluster_ids})

In [ ]:
# create df of bilouvain CoCluster result
from_vertices_v1 = []
to_vertices_v1 = []
from_vertices_v2 = []
to_vertices_v2 = []
f = open(cocluster_res)
for count, line in enumerate(f.readlines()):
    if line[0:2] == "Co":
        good_split = line.split("\n")[0].split(":")[1].split("-")
        from_vertices = int(good_split[0].split("(")[1][:-1])
        to_vertices = int(good_split[1])
        if good_split[0][0:2] == "V1":
            from_vertices_v1.append(from_vertices)
            to_vertices_v1.append(to_vertices)
        if good_split[0][0:2] == "V2":
            from_vertices_v2.append(from_vertices)
            to_vertices_v2.append(to_vertices)
f.close()

# checking if V1 - CoClustering is the same as V2
df_v1 = pd.DataFrame({"Phenotype Cluster V1": to_vertices_v1, "Loci Cluster V1": from_vertices_v1})
df_v2 = pd.DataFrame({"from_Cluster V2": from_vertices_v2, "to_Cluster V2": to_vertices_v2})


sort_v1 = df_v1.sort_values(by="Phenotype Cluster V1")
sort_v1.rename(columns={"Loci Cluster V1": "col2", "Phenotype Cluster V1": "col1"}, inplace=True)

sort_v2 = df_v2.sort_values(by="from_Cluster V2")
sort_v2.rename(columns={"from_Cluster V2": "col1", "to_Cluster V2": "col2"}, inplace=True)

sort_v1.reset_index(drop=True, inplace=True)
sort_v2.reset_index(drop=True, inplace=True)
sort_v1.compare(sort_v2) ## the dataframe are the exact same - therefore we can use df_v1 or df_v2 to get information about CoCluster

# adding +1 to CoCluster ids, more understandable
df_v1["CoCluster ID"] = [x+1 for x in df_v1.index]

In [ ]:
# add bilouvain clustering information (CoCluster_ID, phenotypes in cluster and loci in cluster) to df_loci_info dataframe

pheno_cluster = []
loci_cluster = []
co_cluster = []
for index, entry in df_loci_info.iterrows():
    pheno_id = entry.id
    locus_id = entry["locus_id.r2"]
    
    pheno_cluster_num = df_communities[df_communities["entry"]==pheno_id].Community.values[0]
    pheno_cluster.append(pheno_cluster_num)
    loci_cluster_num = df_communities[df_communities["entry"]==locus_id].Community.values[0]
    loci_cluster.append(loci_cluster_num)
    
    # if the connection of loci-cluster to phenotype-cluster given save CoCluster ID, else save nan 
    try:
        co_cluster.append(df_v1[(df_v1["Phenotype Cluster V1"]==pheno_cluster_num) & (df_v1["Loci Cluster V1"]==loci_cluster_num)]["CoCluster ID"].values[0])
    except:
        co_cluster.append(np.nan)

df_loci_info["CoCluster_ID"] = co_cluster
df_loci_info["phenotype_cluster"] = pheno_cluster
df_loci_info["locus_cluster"]= loci_cluster

In [ ]:
## for enrichment analysis & further analysis create dataframe which does not include nan's in column: CoCluster_ID 
df_loci_info_all = df_loci_info

df_loci_info = df_loci_info.dropna(subset=["CoCluster_ID"]).reset_index(drop=True)

In [ ]:
#checking if all loci with same id are in the same cluster
for locus_id in np.unique(df_loci_info["locus_id.r2"]):
    loci_cluster_ = []
    for index, entry in df_loci_info[df_loci_info["locus_id.r2"]==locus_id].iterrows():
        #print(entry)
        loci_cluster_.append(entry["locus_cluster"])
    if len(np.unique(loci_cluster_)) != 1:
        print(locus_id)

#-> nothing printed out: therefore, all loci with the same id in one cluster

In [ ]:
# enrichment analysis within CoClusters
# gprofiler uses dataframe, which needs Cocluster and candidate genes within this cluster (provided in df_loci_info) 

## uncomment if you want to save
df_loci_info.to_csv(for_loci_enrich_path, index=False)
df_loci_info_all.to_csv(for_pheno_enrich_path)

## create overview of CoCluster dataframe (containing: phenotype, phenotype description, gene names and locus ids)

In [ ]:
def retrieve_information_from_df(df, cluster_id, column_to_match_ids):
    pheno_category = []
    pheno_description = []

    pheno_ids = []
    locus_ids = []
    
    gene_names = []
    
    for index, row in df[df[column_to_match_ids]==cluster_id].iterrows():
        pheno_ids.append(row["id"])
        pheno_category.append(row["category"])
        pheno_description.append(row["phenotype"])
        locus_ids.append(row["locus_id.r2"])

        if not pd.isnull(row["candidate.gene.locus.r2"]):
            split = row["candidate.gene.locus.r2"].split("|")
            if len(split) > 1:
                for entry in split:
                    print(entry)
                    gene_names.append(entry)
            else:
                gene_names.append(split[0])
    
    gene_names = np.unique(gene_names)        

    return(pheno_ids, locus_ids, pheno_category, pheno_description, gene_names)

In [ ]:
phenotypes_description = []

phenotypes_as_string = []
num_of_phenotypes = []

category_names = []
num_of_categories = []

loci_as_string = []
num_of_loci = []

loci_clusters_ids = []
pheno_clusters_ids = []

num_of_genes = []
genes_string = []

for cocluster_id in np.unique(df_loci_info["CoCluster_ID"]):
    
    pheno_ids, locus_ids, pheno_category, pheno_description, gene_names = retrieve_information_from_df(df_loci_info, cocluster_id, "CoCluster_ID")
    
    pheno_ids_string =  ",".join(str(x) for x in np.unique(pheno_ids)) 
    num_of_phenotypes.append(len(np.unique(pheno_ids)))
    phenotypes_as_string.append(pheno_ids_string)
    
    
    pheno_category_string = ",".join(str(x) for x in  np.unique(pheno_category))
    category_names.append(pheno_category_string)
    num_of_categories.append(len(np.unique(pheno_category)))
    
    
    pheno_description = ",".join(str(x) for x in np.unique(pheno_description))
    phenotypes_description.append(pheno_description)
    
    locus_ids_string = ",".join(str(x) for x in np.unique(locus_ids))
    loci_as_string.append(locus_ids_string)
    num_of_loci.append(len(np.unique(locus_ids)))

    loci_clusters_ids.append(df_loci_info[df_loci_info["CoCluster_ID"]==cocluster_id]["locus_cluster"].values[0])
    pheno_clusters_ids.append(df_loci_info[df_loci_info["CoCluster_ID"]==cocluster_id]["phenotype_cluster"].values[0])
    
    
    num_of_genes.append(len(gene_names))
    genes_string.append("|".join(gene_names))

In [ ]:
sup_table = pd.DataFrame({"CoCluster ID": np.unique(df_loci_info["CoCluster_ID"]), 
              "Phenotype": phenotypes_as_string, 
              "Phenotype Description": phenotypes_description,
              "Category of Phenotypes": category_names,
              "Loci": loci_as_string,
              #"Loci Description": locis_description,
              "Number of Phenotypes": num_of_phenotypes,
              "Number of Loci": num_of_loci,
              
              "Number of categories": num_of_categories,
              "Candidate genes": genes_string,
              "Number of candidate genes": num_of_genes,
              "Phenotype Community ID": pheno_clusters_ids,
              "Loci Community ID": loci_clusters_ids,   
             })

## uncomment if you want to save
sup_table.to_csv(overview_CoCluster, index=False)

# 04 run R script for gene enrichment analysis (gprofiler)

# 05 Analysis gene enrichment analysis

In [ ]:
df = pd.read_csv(enrichment_results_loci, sep="\t")
df_pheno = pd.read_csv(enrichment_results_pheno, sep="\t")

In [ ]:
# bring together loci & pheno enrichment results

df_new = pd.DataFrame
for index, row in df_v1.iterrows():
    loci_cluster_info = df[df["Cluster"]== row["Loci Cluster V1"]]
    pheno_cluster_info = df_pheno[df_pheno["Cluster"]== row["Phenotype Cluster V1"]]
    #cocluster_line = row["CoCluster ID"]
    t = pd.concat([loci_cluster_info.drop(columns=["Cluster", "query"]).reset_index(), pheno_cluster_info.drop(columns=["Cluster", "query"]).reset_index()], ignore_index=True)
    
    # remove all duplicates here (so that information is only present once)
    t = t.sort_values(by="p_value", ascending=False).drop_duplicates(subset=["term_name"], keep="first")
    
    cocluster_line = np.repeat(row["CoCluster ID"], len(t))
    t["CoCluster ID"] = cocluster_line

    if not df_new.empty:
        df_new = pd.concat([df_new, t])
    else:
        df_new = t

df_new = df_new.drop(columns=["index"]).reset_index(drop=True) # contains all pheno and loci enrichments (that could be found in each Phenotype and Loci Cluster) per CoCluster

In [ ]:
df_new.to_csv(final_pheno_loci_df_path)